**IMPORT IMPORTANT LIBRARIES**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_rows", None, "display.max_columns", None)

import random
import os
# :seed to make all processes deterministic
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

SEED = 42
seed_everything(SEED)

In [2]:
from datetime import datetime
from sklearn.model_selection import train_test_split,GridSearchCV,KFold,StratifiedKFold,StratifiedShuffleSplit,ShuffleSplit
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler,RobustScaler,PolynomialFeatures,StandardScaler,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor,StackingRegressor,VotingRegressor
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.base import TransformerMixin
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import random
import os
import sklearn

from sklearn.metrics import f1_score


**GET PATIENT HISTORICAL DATA**

Here, I upload these historical data from my google drive. In this demo, encounters.csv is used.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
sample_data=pd.read_csv('/content/drive/MyDrive/generated_data/csv/encounters.csv')

In [5]:
sample_data.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,8d3b00ea-681f-4f18-9e62-8d2ee98aae1a,1978-09-24T16:45:53Z,1978-09-24T17:00:53Z,c3b2e799-5291-dc30-dbfc-679181de00aa,ccb5b561-e2b3-365c-a430-daeb9e4ea5eb,843403b7-6284-3663-9fd0-23f86dae397c,e03e23c9-4df1-3eb6-a62d-f70f02301496,wellness,410620009,Well child visit (procedure),146.76,939.64,0.0,NaN,NaN
1,3d45be6c-b27f-c771-7bba-ccc85ed702ea,1978-10-29T16:45:53Z,1978-10-29T17:00:53Z,c3b2e799-5291-dc30-dbfc-679181de00aa,ccb5b561-e2b3-365c-a430-daeb9e4ea5eb,843403b7-6284-3663-9fd0-23f86dae397c,e03e23c9-4df1-3eb6-a62d-f70f02301496,wellness,410620009,Well child visit (procedure),146.76,1155.52,0.0,NaN,NaN
2,43a466cb-d4b5-9e25-bc8c-123a09f13aa3,1978-12-31T16:45:53Z,1978-12-31T17:00:53Z,c3b2e799-5291-dc30-dbfc-679181de00aa,ccb5b561-e2b3-365c-a430-daeb9e4ea5eb,843403b7-6284-3663-9fd0-23f86dae397c,e03e23c9-4df1-3eb6-a62d-f70f02301496,wellness,410620009,Well child visit (procedure),146.76,418.76,0.0,NaN,NaN
3,5222266e-675d-56a2-44ce-5b52d3aa1b42,1979-12-02T16:45:53Z,1979-12-02T17:00:53Z,c3b2e799-5291-dc30-dbfc-679181de00aa,ccb5b561-e2b3-365c-a430-daeb9e4ea5eb,843403b7-6284-3663-9fd0-23f86dae397c,e03e23c9-4df1-3eb6-a62d-f70f02301496,wellness,410620009,Well child visit (procedure),146.76,418.76,0.0,NaN,NaN
4,c623c695-2884-6041-8e0a-d76ba2b88217,1981-03-01T16:45:53Z,1981-03-01T17:00:53Z,c3b2e799-5291-dc30-dbfc-679181de00aa,ccb5b561-e2b3-365c-a430-daeb9e4ea5eb,843403b7-6284-3663-9fd0-23f86dae397c,e03e23c9-4df1-3eb6-a62d-f70f02301496,wellness,410620009,Well child visit (procedure),146.76,282.76,0.0,NaN,NaN


**GET PCS CODES AND CODE DESCRIPTIONS FROM DRIVE**

In [6]:
file = open("/content/drive/MyDrive/icd10pcs_codes_2023.txt", "r")
pcs_code=[]
pcs_description=[]
while True:
  content=file.readline()[:-1]
  content_list=content.split(" ")
  pcs_code.append(content_list[0])
  pcs_description.append(' '.join(word for word in content_list[1:]))
  if not content:
    break
  print(content)
file.close()

Streaming output truncated to the last 5000 lines.
0U548ZZ Destruction of Uterine Supporting Structure, Via Natural or Artificial Opening Endoscopic
0U550ZZ Destruction of Right Fallopian Tube, Open Approach
0U553ZZ Destruction of Right Fallopian Tube, Percutaneous Approach
0U554ZZ Destruction of Right Fallopian Tube, Percutaneous Endoscopic Approach
0U557ZZ Destruction of Right Fallopian Tube, Via Natural or Artificial Opening
0U558ZZ Destruction of Right Fallopian Tube, Via Natural or Artificial Opening Endoscopic
0U560ZZ Destruction of Left Fallopian Tube, Open Approach
0U563ZZ Destruction of Left Fallopian Tube, Percutaneous Approach
0U564ZZ Destruction of Left Fallopian Tube, Percutaneous Endoscopic Approach
0U567ZZ Destruction of Left Fallopian Tube, Via Natural or Artificial Opening
0U568ZZ Destruction of Left Fallopian Tube, Via Natural or Artificial Opening Endoscopic
0U570ZZ Destruction of Bilateral Fallopian Tubes, Open Approach
0U573ZZ Destruction of Bilateral Fallopian Tub

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



**CONVERT PCS CODES TO DATAFRAME**

In [7]:
df_pcs=pd.DataFrame({"descriptions": pcs_description, "code": pcs_code})
df_pcs.to_csv("pcs_codes_df.csv", index=False)

In [8]:
df_pcs.head()

,descriptions,code
0,Bypass Cerebral Ventricle to Nasopharynx with ...,0016070
1,Bypass Cerebral Ventricle to Mastoid Sinus wit...,0016071
2,Bypass Cerebral Ventricle to Atrium with Autol...,0016072
3,Bypass Cerebral Ventricle to Blood Vessel with...,0016073
4,Bypass Cerebral Ventricle to Pleural Cavity wi...,0016074


**GET CM CODES AND DESCRIPTIONS FROM DRIVE**

In [9]:
file = open("/content/drive/MyDrive/icd10cm-codes-2024.txt", "r")
cm_code=[]
cm_description=[]
while True:
  content=file.readline()[:-1]
  content_list=content.split(" ")
  cm_code.append(content_list[0])
  cm_description.append(' '.join(word for word in content_list[1:]))
  if not content:
    break
  print(content)
file.close()

Streaming output truncated to the last 5000 lines.
S49029A Salter-Harris Type II physeal fracture of upper end of humerus, unspecified arm, initial encounter for closed fracture
S49029D Salter-Harris Type II physeal fracture of upper end of humerus, unspecified arm, subsequent encounter for fracture with routine healing
S49029G Salter-Harris Type II physeal fracture of upper end of humerus, unspecified arm, subsequent encounter for fracture with delayed healing
S49029K Salter-Harris Type II physeal fracture of upper end of humerus, unspecified arm, subsequent encounter for fracture with nonunion
S49029P Salter-Harris Type II physeal fracture of upper end of humerus, unspecified arm, subsequent encounter for fracture with malunion
S49029S Salter-Harris Type II physeal fracture of upper end of humerus, unspecified arm, sequela
S49031A Salter-Harris Type III physeal fracture of upper end of humerus, right arm, initial encounter for closed fracture
S49031D Salter-Harris Type III physeal fr

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




S86111S Strain of other muscle(s) and tendon(s) of posterior muscle group at lower leg level, right leg, sequela
S86112A Strain of other muscle(s) and tendon(s) of posterior muscle group at lower leg level, left leg, initial encounter
S86112D Strain of other muscle(s) and tendon(s) of posterior muscle group at lower leg level, left leg, subsequent encounter
S86112S Strain of other muscle(s) and tendon(s) of posterior muscle group at lower leg level, left leg, sequela
S86119A Strain of other muscle(s) and tendon(s) of posterior muscle group at lower leg level, unspecified leg, initial encounter
S86119D Strain of other muscle(s) and tendon(s) of posterior muscle group at lower leg level, unspecified leg, subsequent encounter
S86119S Strain of other muscle(s) and tendon(s) of posterior muscle group at lower leg level, unspecified leg, sequela
S86121A Laceration of other muscle(s) and tendon(s) of posterior muscle group at lower leg level, right leg, initial encounter
S86121D Laceration o

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
V940XXD Hitting object or bottom of body of water due to fall from watercraft, subsequent encounter
V940XXS Hitting object or bottom of body of water due to fall from watercraft, sequela
V9411XA Bather struck by powered watercraft, initial encounter
V9411XD Bather struck by powered watercraft, subsequent encounter
V9411XS Bather struck by powered watercraft, sequela
V9412XA Bather struck by nonpowered watercraft, initial encounter
V9412XD Bather struck by nonpowered watercraft, subsequent encounter
V9412XS Bather struck by nonpowered watercraft, sequela
V9421XA Rider of nonpowered watercraft struck by other nonpowered watercraft, initial encounter
V9421XD Rider of nonpowered watercraft struck by other nonpowered watercraft, subsequent encounter
V9421XS Rider of nonpowered watercraft struck by other nonpowered watercraft, sequela
V9422XA Rider of nonpowered watercraft struck by powered watercraft, initial encounter
V9422XD Rider of nonp

In [10]:
df_cm=pd.DataFrame({"descriptions": cm_description, "code": cm_code})
df_cm.to_csv("cm_codes_df.csv", index=False)

In [11]:
df_cm.head()

,descriptions,code
0,"Cholera due to Vibrio cholerae 01, biovar c...",A000
1,"Cholera due to Vibrio cholerae 01, biovar e...",A001
2,"Cholera, unspecified",A009
3,"Typhoid fever, unspecified",A0100
4,Typhoid meningitis,A0101


**CONCATENATE PCS AND CM CODES**

In [12]:
new_train=pd.concat([df_pcs, df_cm])
new_train.columns = ["text", "labels"]
new_train.tail(5)

,text,labels
74040,Dependence on other enabling machines and de...,Z9989
74041,Vaping-related disorder,U070
74042,COVID-19,U071
74043,"Post COVID-19 condition, unspecified",U099
74044,,


**SAVE NEW_TRAIN FILE**

In [ ]:
new_train.to_csv('combined_codes.csv', index=False)

**CREATE AN INTEGER-CODE DICTIONARY FOR THE CODES**

In [13]:
temp_list = new_train["labels"].tolist()

# dictionary that maps integer to its string value
label_dict = {}

# list to store integer labels
int_labels = []

for i in range(len(temp_list)):
    label_dict[i] = temp_list[i]
    int_labels.append(i)

In [ ]:
new_train["labels"]=int_labels

In [ ]:
new_train.head()

,text,labels
0,Bypass Cerebral Ventricle to Nasopharynx with ...,0
1,Bypass Cerebral Ventricle to Mastoid Sinus wit...,1
2,Bypass Cerebral Ventricle to Atrium with Autol...,2
3,Bypass Cerebral Ventricle to Blood Vessel with...,3
4,Bypass Cerebral Ventricle to Pleural Cavity wi...,4


**COMPUTE SENTENCE EMBEDDING SIMILARITIES AND GENERATE TOP N PREDICTIONS**

In [14]:
!pip install -U sentence-transformers

from sentence_transformers import SentenceTransformer, util
# sentence = sample_data['DESCRIPTION'][0]
to_compare = new_train['text'].to_list()

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')






     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4747e7d2d123e92a3aef9ef8be758c2f913418d197b52db8270cd1d96c65b6af
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [15]:
sentence='General Appearance: WDWN patient in NAD Skin: Intact Head: contusion frontal bone Eyes: PERRL, EOMI, Sclera, Clear bilaterally, fundi normal bilateral Ears: TMs, normal bilaterally Nose: No congestion or discharge Throat: MMM, no erythema or lesions Neck: Supple, No cervical LAD Cardiovascular System: RRR, no murmur, gallop, or rub Respiratory System: CTAB, normal rate, rhythm, and effort Abdomen: Non-tender, non-distended, soft, +BS, no HSM Extremities: No cyanosis, clubbing, or edema bilaterally. Lab_orders:In-House Labs:Influenza;In-House Labs:Strep. Treatment notes: Plan finish Suprax Return to school Monday Follow Up: PRN. Chief complaints: wet cough with low grade fever'

In [16]:
#Compute embedding for both lists
embedding_1= model.encode(sentence, convert_to_tensor=True)




In [17]:
embeddings = [model.encode(i, convert_to_tensor=True) for i in to_compare]

In [18]:
similarities=[util.pytorch_cos_sim(embedding_1, i) for i in embeddings]


In [19]:
def save_index(similarity_list):
  similarity_dict={}
  for i,j in enumerate(similarity_list):
    similarity_dict[j]=i
  return similarity_dict



In [20]:
def top_n_codes(n):
  for j in sorted(similarities)[::-1][:n]:
    print(label_dict[save_index(similarities)[j]])
    print(new_train.iloc[save_index(similarities)[j], 0])



In [22]:
top_n_codes(10)

S100XXS
Contusion of throat, sequela
R058
   Other specified cough
J399
   Disease of upper respiratory tract, unspecified
J0430
  Supraglottitis, unspecified, without obstruction
R051
   Acute cough
J0431
  Supraglottitis, unspecified, with obstruction
J989
   Respiratory disorder, unspecified
H68009
 Unspecified Eustachian salpingitis, unspecified ear
J22
    Unspecified acute lower respiratory infection
H15003
 Unspecified scleritis, bilateral
